In [2]:
import pandas as pd 
import numpy as np 

In [16]:
OrthGroup=pd.read_csv(
    "/public/home/yllai/songyang/representative/A_D/OrthoFinder/Results_May13/Orthogroups/Orthogroups.GeneCount.tsv",
    header=0,index_col=0,sep="\t"
)
OrthGroup['AD1']=OrthGroup[['AD1_A','AD1_D']].apply(np.sum,axis=1)
OrthGroup['AD2']=OrthGroup[['AD2_A','AD2_D']].apply(np.sum,axis=1)
OrthGroup['AD3']=OrthGroup[['AD3_A','AD3_D']].apply(np.sum,axis=1)
OrthGroup['AD4']=OrthGroup[['AD4_A','AD4_D']].apply(np.sum,axis=1)
OrthGroup['AD5']=OrthGroup[['AD5_A','AD5_D']].apply(np.sum,axis=1)
OrthGroup_filter=OrthGroup[['A1','A2','AD1','AD2','AD3','AD4','AD5','D5']].applymap(
    lambda x: 1 if x>=1 else 0)
OrthGroup_filter    

,A1,A2,AD1,AD2,AD3,AD4,AD5,D5
Orthogroup,,,,,,,,
OG0000000,1,1,1,1,1,1,1,1
OG0000001,1,1,1,1,1,1,1,1
OG0000002,1,1,1,1,1,1,1,1
OG0000003,1,1,1,1,1,1,1,1
OG0000004,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
OG0045718,0,0,0,0,0,0,0,1
OG0045719,0,0,0,0,0,0,0,1
OG0045720,0,0,0,0,0,0,0,1


In [50]:
OrthGroupType=OrthGroup_filter.apply(lambda x:"".join([str(i) for i in x]),axis=1)
OrthGroupTypeCount=OrthGroupType.value_counts()
#* 筛选OrthGroup大于500的类型
filterOrthGroup=OrthGroupType[OrthGroupType.isin(OrthGroupTypeCount[OrthGroupTypeCount.apply(lambda x:x>=500)].index)].index
#* 绘制upset图数据
OrthGroup_filter.loc[filterOrthGroup].to_csv("upsetVenn.txt",header=True,index=True,sep="\t")

In [57]:
#----------------------------------------------
#TODO： 将基因family分类
#----------------------------------------------
def OrthGroupCat(x):
    geneCount=len([i for i in x if i==1])
    if geneCount==8:
        return "Required"
    elif geneCount==7:
        return "Soft"
    elif geneCount>=2 and geneCount<=6:
        return "Variable"
    else:
        return "Specific"
OrthGroup['OrthGroupType']=OrthGroup_filter.apply(OrthGroupCat,axis=1)
OrthGroup.to_csv("./Orthogroups_GeneCount.txt",header=True,index=True,sep="\t")

In [64]:
#---------------------------------------------------------
#TODO 将基因位置绘制在染色体上
#---------------------------------------------------------
OrthGroupGeneId=pd.read_csv("/public/home/yllai/songyang/representative/A_D/OrthoFinder/Results_May13/Orthogroups/Orthogroups.tsv",
    header=0,index_col=0,sep="\t"
)

In [104]:
geneData=[]
for row,geneCate in OrthGroupGeneId[['D5']].iterrows():
    geneType=OrthGroup.loc[row]['OrthGroupType']
    for geneCate_V2 in geneCate.dropna().values:
        for geneId in geneCate_V2.split(","):
            geneData.append(
                (row,geneId.strip(),geneType)
            )
pd.DataFrame(geneData,columns=['OrthoGroup','GeneId','OrthoType']).to_csv(
    "Ortho_genome/D5_OrthoGroup.txt",
    header=True,index=False,sep="\t")

In [125]:
#------------------------------------------------
#* 获取每个基因对应的坐标编号
#------------------------------------------------
import re 
gene_peptide=pd.read_csv("./Circos/AD3/gene_peptide.txt",header=None,index_col=None,sep="\t")
geneLoca=pd.read_csv("./Circos/AD3/gene.txt",header=None,index_col=3,sep="\t")
gene_peptide.index=gene_peptide[1].apply(lambda x:re.sub(r"(\.[0-9]+)$","",x))
pd.concat([geneLoca.loc[gene_peptide.index],gene_peptide],axis=1).to_csv(
    "./Circos/AD3/gene_change.txt",header=False,index=False,sep="\t")

In [157]:
#-------------------------------------------
#TODO： 获取基因对应的类型和基因所在的染色体坐标
#------------------------------------------- 
geneType=pd.read_csv("/public/home/zpliu/Sonyang/Ortho_genome/A2_OrthoGroup.txt",header=0,index_col=None,sep="\t")
geneLocal=pd.read_csv("/public/home/zpliu/Sonyang/Circos/A2/gene.txt",header=None,index_col=3,sep="\t")
RequiredGeneLocal=geneLocal.loc[geneType.loc[geneType['OrthoType']=="Specific"]['GeneId'].apply(lambda x:re.sub(r"(\.[0-9]+)$","",x))]
RequiredGeneLocal[3]=geneType.loc[geneType['OrthoType']=="Specific"]['GeneId'].values
RequiredGeneLocal[4]=geneType.loc[geneType['OrthoType']=="Specific"]['OrthoGroup'].values
RequiredGeneLocal[5]=geneType.loc[geneType['OrthoType']=="Specific"]['OrthoType'].values
RequiredGeneLocal

,0,1,2,3,4,5
3,,,,,,
Garb_04G008510,Chr04,20262436,20263338,Garb_04G008510.1,OG0024111,Specific
Garb_05G034330,Chr05,69477650,69480010,Garb_05G034330.1,OG0024111,Specific
Garb_06G010090,Chr06,29914415,29915372,Garb_06G010090.1,OG0024111,Specific
Garb_06G013420,Chr06,79797309,79798267,Garb_06G013420.1,OG0024111,Specific
Garb_07G005950,Chr07,12489513,12490471,Garb_07G005950.1,OG0024111,Specific
...,...,...,...,...,...,...
Garb_13G021400,Chr13,116921430,116922125,Garb_13G021400.1,OG0040351,Specific
Garb_11G031070,Chr11,113547024,113580808,Garb_11G031070.1,OG0040359,Specific
Garb_13G003950,Chr13,5369176,5399422,Garb_13G003950.1,OG0040359,Specific


In [158]:
RequiredGeneLocal.to_csv("/public/home/zpliu/Sonyang/Circos/A2/specific_gene.txt",header=None,index=False,sep="\t")

In [152]:
geneType['OrthoType'].value_counts()

Required    29165
Variable     8458
Soft         2985
Specific      145
Name: OrthoType, dtype: int64

In [1]:
#---------------------------------------------------
#TODO 分析包含多个基因的Orthologous group在不同棉种中的变化
#---------------------------------------------------
import pandas as pd 
OrthoGroupCount=pd.read_csv("./Orthogroups_GeneCount.txt",header=0,index_col=0,sep="\t")
filterData=OrthoGroupCount.loc[
    OrthoGroupCount['OrthGroupType']=="Required"
    ][['A1','A2','D5','AD1','AD2','AD3','AD4','AD5']]

In [193]:
#-------------------------------------------------
#TODO 在任意物种中基因数目超过10个的为多拷贝的必须基因
#* 对基因的数目进行按行标准化
#-------------------------------------------------
from sklearn import preprocessing
z_scaler = preprocessing.Normalizer()
multipOrthoGneCount=filterData[~filterData.lt(11).all(axis=1)]

geneCoutScale=pd.DataFrame(
    z_scaler.fit_transform(multipOrthoGneCount),
    columns=[['A1','A2','D5','AD1','AD2','AD3','AD4','AD5']])
geneCoutScale.index=multipOrthoGneCount.index    
geneCoutScale.index.name=""
geneCoutScale.to_csv("GeneCount/required_mulGenes.txt",header=True,index=True,sep="\t")

In [10]:
#----------------------------------------------
#* 包含多个基因的非必需Orthologous group在物种间的变化
#----------------------------------------------
from sklearn import preprocessing
filterData=OrthoGroupCount.loc[
    OrthoGroupCount['OrthGroupType']!="Required"
    ][['A1','A2','D5','AD1','AD2','AD3','AD4','AD5']]
multipOrthoGneCount=filterData[~filterData.lt(11).all(axis=1)]    
z_scaler = preprocessing.Normalizer()
multipOrthoGneCount=filterData[~filterData.lt(11).all(axis=1)]

geneCoutScale=pd.DataFrame(
    z_scaler.fit_transform(multipOrthoGneCount),
    columns=[['A1','A2','D5','AD1','AD2','AD3','AD4','AD5']])
geneCoutScale.index=multipOrthoGneCount.index    
geneCoutScale.index.name=""
geneCoutScale.to_csv("GeneCount/norequired_mulGenes.txt",header=True,index=True,sep="\t")

In [2]:
def geneCategor(x):
    A1,A2,D5,At,Dt=x
    if ((A1!=0 or A2!=0) and D5!=0 and At!=0 and Dt!=0):
        return 'conserved'
    elif ((A1==0 and A2==0) and D5!=0 and At!=0 and Dt!=0):
        return "At_gain"
    elif ((A1!=0 or A2!=0) and D5==0 and At!=0 and Dt!=0):
        return "Dt_gain"
    elif ((A1==0 and A2==0) and D5!=0 and At==0 and Dt!=0):
        return "onlyDt"
    elif ((A1!=0 or A2!=0) and D5==0 and At!=0 and Dt==0):
        return "onlyAt"
    elif ((A1!=0 or A2!=0) and D5!=0 and At==0 and Dt!=0):
        return "A2_loss"
    elif ((A1!=0 or A2!=0) and D5!=0 and At!=0 and Dt==0):
        return "D5_loss"
    else:
        return "other"

In [3]:
AD1=OrthoGroupCount[['A1','A2','D5','AD1_A','AD1_D']]
AD2=OrthoGroupCount[['A1','A2','D5','AD2_A','AD2_D']]
AD3=OrthoGroupCount[['A1','A2','D5','AD3_A','AD3_D']]
AD4=OrthoGroupCount[['A1','A2','D5','AD4_A','AD4_D']]
AD5=OrthoGroupCount[['A1','A2','D5','AD5_A','AD5_D']]
#* 统计每个四倍体中基因family的变化情况
AD1_category=AD1.apply(geneCategor,axis=1)
AD2_category=AD2.apply(geneCategor,axis=1)
AD3_category=AD3.apply(geneCategor,axis=1)
AD4_category=AD4.apply(geneCategor,axis=1)
AD5_category=AD5.apply(geneCategor,axis=1)

In [125]:
#* 合并各个棉种的数据
MergeData=pd.concat([AD1_category,AD2_category],axis=1)
MergeData=pd.concat([MergeData,AD3_category],axis=1)
MergeData=pd.concat([MergeData,AD4_category],axis=1)
MergeData=pd.concat([MergeData,AD5_category],axis=1)
MergeData.index.name=""
MergeData.columns=['AD1','AD2','AD3','AD4','AD5']
pd.melt(MergeData,value_vars=['AD1','AD2','AD3','AD4','AD5']).to_csv("GeneCount/polyploid.txt",header=True,index=False,sep="\t")
#---------------------------------------------------
#* 整理桑基图数据
#---------------------------------------------------
MergeData['merge']=MergeData.apply(lambda x: "-".join(x),axis=1)
plotData=pd.DataFrame(MergeData['merge'].value_counts())
plotData['Type']=plotData.index
MergeData=pd.concat([plotData['Type'].str.split("-",expand=True),plotData[['merge']]],axis=1)
MergeData.columns=['AD1','AD2','AD3','AD4','AD5','Freq']

In [123]:
MergeData.to_csv("./GeneCount/sankey.txt",header=True,index=False,sep="\t")

In [127]:
#* 所有OrthoGroup的保守情况
MergeData.to_csv("./GeneCount/All_OrthoGroup_conserved.txt",header=True,index=True,sep="\t")